deterministic action

In [ ]:
import gym
import universe

In [ ]:
import math

import time
import random
import pickle

from collections import deque

%matplotlib notebook
import matplotlib as mpl
plt = mpl.pyplot
# mpl.pylab.rcParams['figure.figsize'] = (12, 9)

import numpy as np
import torch as tch
F = tch.nn.functional

In [ ]:
# pytorch auxiliar functions
def np2var(input):
    output = tch.autograd.Variable(tch.from_numpy(input))
    if tch.cuda.is_available():
        output = output.cuda()
    return output

tanh2sigmoid = lambda x: (x + 1) / 2

def postprocess(tch_img):
    return (tch_img * 255).byte().data.cpu().numpy().squeeze()

# screen slicing00
top = 86
left = 20 + 100
height = 300
width = 500 - 200
gamescreen_slice = (slice(top,  top + height), slice(left, left + width))
game_center = (top + height // 2, left + width // 2) # y, x
game_center = game_center[::-1]

# average pooling for downscaling
s = downscale_size = 2
pool2d_downscale = tch.nn.AvgPool2d((s, s), stride=(s, s))
downscale_n = 1
downscale = downscale_size ** downscale_n
dummy_img = np.zeros((height // downscale, width // downscale, 1))

In [ ]:
# visualization
fig, axes = mpl.pyplot.subplots(1, 3)

# initialization of visualization
imgs = [ax.imshow(dummy_img.squeeze(), interpolation='none', vmin=0, vmax=255, cmap='gray') for ax in axes]

fig_loss, ax_loss = plt.subplots()

In [ ]:
# prevent cluttering
import sys
nb_stdout = sys.stdout
sys.stdout = open('/dev/stdout', 'w')

In [ ]:
env = gym.make('internet.SlitherIO-v0')
env.configure(remotes=1)  # automatically creates a local docker container

# sys.stdout = nb_stdout

In [ ]:
observation_n = env.reset()

total_ae_loss = 0
total_pred_loss = 0

train_steps = 0

hidden = None

action_n = [[('KeyEvent', 'ArrowLeft', random.choice([True, False])),
             ('KeyEvent', 'ArrowRight', random.choice([True, False])),
            ] for ob in observation_n]

episode = []
episode_nr = 397

while episode_nr < 1000:

    # your agent here
    observation_n, reward_n, done_n, info = env.step(action_n)
    
    # every now and then
    if random.random() < 0.01:
        # randomly choose action
        action_n = [[('KeyEvent', 'ArrowLeft', random.choice([True, False])),
                     ('KeyEvent', 'ArrowRight', random.choice([True, False])),
                    ] for ob in observation_n]

    if observation_n[0] is not None:
        # preprocessing
        gamescreen = observation_n[0]['vision'][gamescreen_slice]
        gamescreen = gamescreen.transpose(2, 0, 1)
        gamescreen = np2var(gamescreen).float() / 255
        gamescreen = gamescreen.unsqueeze(0)
        # downscaling
        gamescreen = pool2d_downscale(gamescreen)
        gamescreen = gamescreen.max(dim=1, keepdim=True)[0]
            
        

        #displaying
        gamescreen = postprocess(gamescreen)
#             decoded_gamescreen = postprocess(decoded_gamescreen)
#             decoded_pred_gamescreen = postprocess(decoded_pred_gamescreen)
        imgs[0].set_data(gamescreen)
#             imgs[1].set_data(decoded_gamescreen)
#             imgs[2].set_data(decoded_pred_gamescreen)
        fig.canvas.draw()
    
        episode.append([gamescreen, reward_n])

    if np.any(done_n):
        if len(episode) > 32:
            pickle.dump(episode, open('./game_dataset/'+str(episode_nr)+'.pickle', 'wb'))
            episode = []
            episode_nr += 1
        

decoder(encoder(train_batch + 1)).max()